In [1]:
# Imports
from tensorflow.keras.layers import TextVectorization
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

In [2]:
# Import data
data = pd.read_csv("../../data/mturk_experiment_2.csv",encoding='unicode_escape')
labels = data["Formality"]
samples = data["Sentence"]

train_samples, test_samples, train_labels,test_labels = train_test_split(samples, labels, test_size=0.2,random_state=5)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [3]:
# Embedding setup
vectoriser = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectoriser.adapt(text_ds)

voc = vectoriser.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

glove_path = "glove.6B.100d.txt"
embeddings_index = {}
with open(glove_path,encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs,"f",sep= " ")
        embeddings_index[word] = coefs

num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens,embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

embedding_layer = Embedding(num_tokens,embedding_dim,embeddings_initializer=keras.initializers.Constant(embedding_matrix),trainable=False)

In [4]:
# CNN setup
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(units=1)(x)
cnn = keras.Model(int_sequences_input, preds)
cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])

In [5]:
# LSTM setup

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.LSTM(4,input_shape=(1,5))(embedded_sequences)
out = layers.Dense(1,activation='relu')(x)

lstm = keras.Model(int_sequences_input,out)
lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])

In [6]:
# RNN setup
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.GRU(256,return_sequences=True)(embedded_sequences)
x = layers.SimpleRNN(128)(x)
out = layers.Dense(1,activation='relu')(x)

rnn = keras.Model(int_sequences_input,out)
rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])

In [7]:
# Data setup
x_train = vectoriser(np.array([[s] for s in train_samples])).numpy()
x_val = vectoriser(np.array([[s] for s in test_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(test_labels)

results = pd.DataFrame()

In [8]:
# Try CNN with batch sizes 10,32 and epochs 20,50,75 and 100
#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=10,epochs=20)
results["CNN 20 B10"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=32,epochs=20)
results["CNN E20 B32"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=10,epochs=100)
results["CNN E100 B10"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=32,epochs=100)
results["CNN E100 B32"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=10,epochs=75)
results["CNN E75 B10"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=32,epochs=75)
results["CNN E75 B32"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=10,epochs=50)
results["CNN E50 B10"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=32,epochs=50)
results["CNN E50 B32"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

Epoch 1/20
563/563 [==============================] - 5s 8ms/step - loss: 2.0521 - mean_absolute_error: 1.0854 - mean_absolute_percentage_error: 28.5907
Epoch 2/20
563/563 [==============================] - 4s 8ms/step - loss: 1.0460 - mean_absolute_error: 0.8166 - mean_absolute_percentage_error: 22.1431
Epoch 3/20
563/563 [==============================] - 4s 7ms/step - loss: 0.9423 - mean_absolute_error: 0.7797 - mean_absolute_percentage_error: 21.2027
Epoch 4/20
563/563 [==============================] - 5s 8ms/step - loss: 0.9430 - mean_absolute_error: 0.7766 - mean_absolute_percentage_error: 21.0305
Epoch 5/20
563/563 [==============================] - 6s 10ms/step - loss: 0.8951 - mean_absolute_error: 0.7575 - mean_absolute_percentage_error: 20.5990
Epoch 6/20
563/563 [==============================] - 8s 13ms/step - loss: 0.8928 - mean_absolute_error: 0.7540 - mean_absolute_percentage_error: 20.4318
Epoch 7/20
563/563 [==============================] - 8s 13ms/step - loss: 0.875

In [9]:
results

,CNN 20 B10,CNN E20 B32,CNN E100 B10,CNN E100 B32,CNN E75 B10,CNN E75 B32,CNN E50 B10,CNN E50 B32
0,0.710439,0.685374,0.821238,0.857994,0.819501,0.818001,0.809370,0.803575
1,0.671222,0.657621,0.715901,0.729447,0.713008,0.714696,0.713086,0.707625
2,17.222521,17.496559,19.365547,19.448305,19.347984,19.389345,19.679512,19.351738


In [10]:
# Try LSTM with batch sizes 10,32 and epochs 20,50,75 and 100
#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=10,epochs=20)
results["LSTM E20 B10"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=32,epochs=20)
results["LSTM E20 B32"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=10,epochs=100)
results["LSTM E100 B10"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=32,epochs=100)
results["LSTM E100 B32"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=10,epochs=75)
results["LSTM E75 B10"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=32,epochs=75)
results["LSTM E75 B32"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=10,epochs=50)
results["LSTM E50 B10"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=32,epochs=50)
results["LSTM E50 B32"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

Epoch 1/20
563/563 [==============================] - 56s 94ms/step - loss: 16.7411 - mean_absolute_error: 3.9282 - mean_absolute_percentage_error: 96.1022
Epoch 2/20
563/563 [==============================] - 54s 95ms/step - loss: 8.4907 - mean_absolute_error: 2.6724 - mean_absolute_percentage_error: 62.1918
Epoch 3/20
563/563 [==============================] - 54s 95ms/step - loss: 5.5817 - mean_absolute_error: 2.0922 - mean_absolute_percentage_error: 47.2302
Epoch 4/20
563/563 [==============================] - 53s 95ms/step - loss: 4.2248 - mean_absolute_error: 1.7776 - mean_absolute_percentage_error: 39.8861
Epoch 5/20
563/563 [==============================] - 54s 95ms/step - loss: 3.2995 - mean_absolute_error: 1.5430 - mean_absolute_percentage_error: 34.8409
Epoch 6/20
563/563 [==============================] - 76s 135ms/step - loss: 2.6262 - mean_absolute_error: 1.3601 - mean_absolute_percentage_error: 31.3685
Epoch 7/20
563/563 [==============================] - 53s 95ms/step 

In [11]:
results

,CNN 20 B10,CNN E20 B32,CNN E100 B10,CNN E100 B32,CNN E75 B10,CNN E75 B32,CNN E50 B10,CNN E50 B32,LSTM E20 B10,LSTM E20 B32,LSTM E100 B10,LSTM E100 B32,LSTM E75 B10,LSTM E75 B32,LSTM E50 B10,LSTM E50 B32
0,0.710439,0.685374,0.821238,0.857994,0.819501,0.818001,0.809370,0.803575,1.259489,1.259467,1.259567,1.259547,1.259134,1.259371,1.259519,1.259608
1,0.671222,0.657621,0.715901,0.729447,0.713008,0.714696,0.713086,0.707625,0.935931,0.935941,0.935900,0.935908,0.936091,0.935981,0.935919,0.935884
2,17.222521,17.496559,19.365547,19.448305,19.347984,19.389345,19.679512,19.351738,28.366518,28.364433,28.373669,28.371843,28.330305,28.355181,28.369255,28.377270


In [12]:
# Try RNN with batch sizes 10,32 and epochs 50,75 and 100
#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=10,epochs=100)
results["RNN E100 B10"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=32,epochs=100)
results["RNN E100 B32"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=10,epochs=75)
results["RNN E75 B10"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=32,epochs=75)
results["RNN E75 B32"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=10,epochs=50)
results["RNN E50 B10"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=32,epochs=50)
results["RNN E50 B32"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

Epoch 1/100
563/563 [==============================] - 75s 131ms/step - loss: 3.6391 - mean_absolute_error: 1.4127 - mean_absolute_percentage_error: 38.1709
Epoch 2/100
563/563 [==============================] - 74s 131ms/step - loss: 3.2222 - mean_absolute_error: 1.3034 - mean_absolute_percentage_error: 36.0432
Epoch 3/100
563/563 [==============================] - 77s 137ms/step - loss: 3.5960 - mean_absolute_error: 1.3699 - mean_absolute_percentage_error: 37.4957
Epoch 4/100
563/563 [==============================] - 74s 132ms/step - loss: 4.0650 - mean_absolute_error: 1.4559 - mean_absolute_percentage_error: 39.3485
Epoch 5/100
563/563 [==============================] - 74s 132ms/step - loss: 4.6206 - mean_absolute_error: 1.5618 - mean_absolute_percentage_error: 41.6695
Epoch 6/100
563/563 [==============================] - 73s 130ms/step - loss: 5.0901 - mean_absolute_error: 1.6470 - mean_absolute_percentage_error: 43.5629
Epoch 7/100
563/563 [==============================] - 74s

In [13]:
results = results.T
#results.columns = ["MSE","MAE","% Error"]
results.to_csv("../../data/random_state_five/cnn_rnn_scores.csv")